Lấy dữ liệu giá chứng khoán của Ngân hàng VCB từ 1/1/2015 - 24/3/2023 bằng thư viện selenium và BeautifulSoup

In [ ]:
import time
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings("ignore")

In [ ]:
driver = webdriver.Chrome()   # Tạo một phiên bản Chrome WebDriver
driver.get('https://s.cafef.vn/Lich-su-giao-dich-NVL-1.chn')               # Điều hướng đến trang web có đường dẫn URL (trang web sẽ cào dữ liệu)

In [ ]:
def search_symbol(stock, from_day='1/1/2015', to_day='24/3/2023'):
    search = driver.find_element(By.CSS_SELECTOR, '#ContentPlaceHolder1_ctl03_txtKeyword')
    search.clear()
    search.send_keys(stock)   

    from_date = driver.find_element(By.CSS_SELECTOR, '#ContentPlaceHolder1_ctl03_dpkTradeDate1_txtDatePicker')
    from_date.clear()
    from_date.send_keys(from_day)   

    to_date = driver.find_element(By.CSS_SELECTOR, '#ContentPlaceHolder1_ctl03_dpkTradeDate2_txtDatePicker')
    to_date.clear()
    to_date.send_keys(to_day)   

    button_xem = driver.find_element(By.CSS_SELECTOR, '#ContentPlaceHolder1_ctl03_btSearch')
    button_xem.click()

In [ ]:
def crawl_data_one_page(stock):
    tag_table = BeautifulSoup(driver.page_source).find(id='divStart')
    tag_tr = tag_table.find_all('tr')
    idx_next = len(tag_tr[-1].find_all('td'))
    tag_tr = tag_tr[3:-1]
    data = []
    for tr in tag_tr:
        td = tr.find_all('td')
        del td[4], td[-3:]
        tmp = []
        for val in td:
            tmp.append(val.text.replace('\xa0',''))
        data.append(tmp)
        
    return data, idx_next

In [ ]:
data = []
stock = 'VCB'
print(f'crawl data for {stock} stock')
search_symbol(stock)
time.sleep(1)
try:
    i = 1
    while True:
        print(i, end=' ')
        one_page_data, idx_next = crawl_data_one_page(stock)
        data = data + one_page_data

        click_next = driver.find_element(By.CSS_SELECTOR, f'#ContentPlaceHolder1_ctl03_divHO > div > div > table > tbody > tr > td:nth-child({idx_next}) > a')
        click_next.click()
        i +=1
        time.sleep(1)
except:
    pass

crawl data for VCB stock
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 

In [ ]:
col = ['Ngày', 'Giá điều chỉnh', 'Giá đóng cửa', 'Thay đổi (+/-%)', 'GD khớp lệnh_KL', 
       'GD khớp lệnh_GT','GD thỏa thuận_KL','GD thỏa thuận_GT','Giá mở cửa','Giá cao nhất','Giá thấp nhất']
df = pd.DataFrame(data, columns = col)
df.to_csv('./dataset/data_chungkhoan_VCB.csv', index=False)
df

,Ngày,Giá điều chỉnh,Giá đóng cửa,Thay đổi (+/-%),GD khớp lệnh_KL,GD khớp lệnh_GT,GD thỏa thuận_KL,GD thỏa thuận_GT,Giá mở cửa,Giá cao nhất,Giá thấp nhất
0,24/03/2023,89.00,89.00,-1.70 (-1.87 %),"630,000","56,383,000,000","43,000","3,762,500,000",91.00,91.00,88.90
1,23/03/2023,90.70,90.70,1.70 (1.91 %),"849,500","75,853,000,000","1,096,000","97,342,400,000",88.80,90.70,87.30
2,22/03/2023,89.00,89.00,2.50 (2.89 %),"1,055,800","93,158,000,000","43,000","3,848,500,000",86.60,89.90,86.60
3,21/03/2023,86.50,86.50,1.40 (1.65 %),"1,057,400","91,918,000,000","116,000","10,261,000,000",85.40,89.90,85.40
4,20/03/2023,85.10,85.10,-3.90 (-4.38 %),"1,034,000","90,420,000,000","43,000","3,870,000,000",89.00,89.50,85.10
...,...,...,...,...,...,...,...,...,...,...,...
2047,09/01/2015,19.29,36.90,2.30 (6.65 %),"2,286,680","83,558,466,000","247,301","9,052,206,000",34.90,37.00,34.90
2048,08/01/2015,18.09,34.60,1.00 (2.98 %),"904,240","31,296,505,000","100,000","3,469,000,000",33.70,35.00,33.60
2049,07/01/2015,17.56,33.60,-0.40 (-1.18 %),"1,430,340","48,671,177,000","134,000","4,523,562,000",34.10,34.60,33.10
2050,06/01/2015,17.77,34.00,2.10 (6.58 %),"1,684,600","56,234,910,000",0,0,31.90,34.00,31.90
